In [1]:
#eye detection
!pip install dlib
import cv2
import numpy as np
import dlib
cap = cv2.VideoCapture(0)


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# loading face detector 
detector = dlib.get_frontal_face_detector()

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
# blink detection , blinking ratio --> ratio of horizontal to vertical line
from math import hypot
def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))

    hor_line = cv2.line(frame, left_point, right_point, (0, 255, 0), 2)
    ver_line = cv2.line(frame, center_top, center_bottom, (0, 255, 0), 2)

    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    ratio = hor_line_lenght / ver_line_lenght
    return ratio

In [4]:
    def get_gaze_ratio(eye_points,landmarks):              #facial landmarks
        
        left_eye_region = np.array([(landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y),
                            (landmarks.part(eye_points[1]).x, landmarks.part(eye_points[1]).y),
                            (landmarks.part(eye_points[2]).x, landmarks.part(eye_points[2]).y),
                            (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y),
                            (landmarks.part(eye_points[4]).x, landmarks.part(eye_points[4]).y),
                            (landmarks.part(eye_points[5]).x, landmarks.part(eye_points[5]).y)], np.int32)
    
        # creating mask for left eye
        height, width, _ = frame.shape
        mask = np.zeros((height, width), np.uint8)
        cv2.polylines(mask, [left_eye_region], True, 255, 2)
        cv2.fillPoly(mask, [left_eye_region], 255)
        left_eye = cv2.bitwise_and(gray, gray, mask=mask)
        
        #We now extract the eye from the face and we put it on his own window considering all the extremes points of the eye
        min_x = np.min(left_eye_region[:, 0])
        max_x = np.max(left_eye_region[:, 0])
        min_y = np.min(left_eye_region[:, 1])
        max_y = np.max(left_eye_region[:, 1])
        gray_eye = left_eye[min_y: max_y, min_x: max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
        
        
        
        #gaze detection part 2
        gray_eye = left_eye[min_y: max_y, min_x: max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY) #threshold value
        height, width = threshold_eye.shape #to find height of eye
        left_side_threshold = threshold_eye[0: height, 0: int(width / 2)]
        left_side_white = cv2.countNonZero(left_side_threshold) # non zero value->not coloured -> white area
        right_side_threshold = threshold_eye[0: height, int(width / 2): width]
        right_side_white = cv2.countNonZero(right_side_threshold)
        gaze_ratio=0
        if left_side_white==0:
            gaze_ratio=1
        elif right_side_white==0:
            gaze_ratio=5
        else:
            gaze_ratio = left_side_white / right_side_white
        return gaze_ratio

In [ ]:
#while loop that takes frames from webcam



def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)
while True:
    a, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        
        #x, y = face.left(), face.top()
        #x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)
        landmarks = predictor(gray, face)
        left_point = (landmarks.part(36).x, landmarks.part(36).y)
        right_point = (landmarks.part(39).x, landmarks.part(39).y)
        center_top = midpoint(landmarks.part(37), landmarks.part(38))
        center_bottom = midpoint(landmarks.part(41), landmarks.part(40))
       # hor_line = cv2.line(frame, left_point, right_point, (0, 255, 0), 2)
       # ver_line = cv2.line(frame, center_top, center_bottom, (0, 255, 0), 2)
    
    #blink detection
        landmarks = predictor(gray, face)
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
        font = cv2.FONT_HERSHEY_SIMPLEX
        if blinking_ratio > 5.7:
            cv2.putText(frame, "BLINKING", (50, 150), font, 7, (255, 0, 0))
        
        
        
        # gaze detection
        left_eye_region = np.array([(landmarks.part(36).x, landmarks.part(36).y),
                            (landmarks.part(37).x, landmarks.part(37).y),
                            (landmarks.part(38).x, landmarks.part(38).y),
                            (landmarks.part(39).x, landmarks.part(39).y),
                            (landmarks.part(40).x, landmarks.part(40).y),
                            (landmarks.part(41).x, landmarks.part(41).y)], np.int32)
        
        # creating mask for left eye
        height, width, _ = frame.shape
        mask = np.zeros((height, width), np.uint8)
        cv2.polylines(mask, [left_eye_region], True, 255, 2)
        cv2.fillPoly(mask, [left_eye_region], 255)
        left_eye = cv2.bitwise_and(gray, gray, mask=mask)
        
        #We now extract the eye from the face and we put it on his own window considering all the extremes points of the eye
        min_x = np.min(left_eye_region[:, 0])
        max_x = np.max(left_eye_region[:, 0])
        min_y = np.min(left_eye_region[:, 1])
        max_y = np.max(left_eye_region[:, 1])
        gray_eye = left_eye[min_y: max_y, min_x: max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
        
        # we display it on the screen.
        
        #threshold_eye = cv2.resize(threshold_eye, None, fx=5, fy=5)
        #eye = cv2.resize(gray_eye, None, fx=5, fy=5)
        #cv2.imshow("Eye", eye)
        #cv2.imshow("Threshold", threshold_eye)
        #cv2.imshow("Left eye", left_eye)
        
        #gaze detection part 2
        gray_eye = left_eye[min_y: max_y, min_x: max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY) #threshold value
        height, width = threshold_eye.shape #to find height of eye
        left_side_threshold = threshold_eye[0: height, 0: int(width / 2)]
        left_side_white = cv2.countNonZero(left_side_threshold) # non zero value->not coloured -> white area
        right_side_threshold = threshold_eye[0: height, int(width / 2): width]
        right_side_white = cv2.countNonZero(right_side_threshold)
        
        #divide the white pixels of the left part and those of the right part and we get the gaze ratio.
        gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
        gaze_ratio=(gaze_ratio_left_eye + gaze_ratio_right_eye)/2
        
        
        
        #gaze ratio is smaller than 1 when looking to the right side and greater than 1.7 when the eyes are looking to the left side.
        if gaze_ratio <= 1:
            cv2.putText(frame, "RIGHT", (50, 100), font, 7, (255, 0, 0))
            
        elif 1 < gaze_ratio < 1.7:
            cv2.putText(frame, "CENTER", (50, 150), font, 7, (255, 0, 0))
        else:
            
            cv2.putText(frame, "LEFT", (50, 150), font, 7, (255, 0, 0))
        cv2.imshow("Frame", frame)
 
        
        
        keyCode = cv2.waitKey(1)

        if keyCode & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()
